
# Gezinomi Kural Tabanlı Sınıflandırma Projesi

## İş Problemi
Gezinomi yaptığı satışların bazı özelliklerini kullanarak seviye tabanlı (level based) yeni satış tanımları oluşturmak ve bu yeni satış tanımlarına göre segmentler oluşturup bu segmentlere göre yeni gelebilecek müşterilerin şirkete ortalama ne kadar kazandırabileceğini tahmin etmek istemektedir.

**Örnek:**  
Antalya’dan Herşey Dahil bir otele yoğun bir dönemde gitmek isteyen bir müşterinin ortalama ne kadar kazandırabileceği belirlenmek isteniyor.

---

## Veri Seti Hikayesi
`miuul_gezinomi.xlsx` veri seti Gezinomi şirketinin yaptığı satışların fiyatlarını ve bu satışlara ait bilgileri içermektedir. Veri seti her satış işleminde oluşan kayıtlardan meydana gelmektedir. Bunun anlamı tablo tekilleştirilmemiştir. Diğer bir ifade ile müşteri birden fazla alışveriş yapmış olabilir.

---

## Değişkenler
- **SaleId** : Satış id  
- **SaleDate** : Satış Tarihi  
- **Price** : Satış için ödenen fiyat  
- **ConceptName** : Otel konsept bilgisi  
- **SaleCityName** : Otelin bulunduğu şehir bilgisi  
- **CheckInDate** : Müşterinin otele giriş tarihi  
- **CInDay** : Müşterinin otele giriş günü  
- **SaleCheckInDayDiff** : Check-in ile satış tarihi arasındaki gün farkı  
- **Seasons** : Otele giriş tarihindeki sezon bilgisi  

---

## Görevler
1. `miuul_gezinomi.xlsx` dosyasını okutunuz ve veri setiyle ilgili genel bilgileri gösteriniz.  
2. Kaç unique şehir vardır? Frekansları nedir?  
3. Kaç unique Concept vardır?  
4. Hangi Concept’ten kaçar tane satış gerçekleşmiş?  
5. Şehirlere göre satışlardan toplam ne kadar kazanılmış?  
6. Concept türlerine göre ne kadar kazanılmış?  
7. Şehirlere göre Price ortalamaları nedir?  
8. Conceptlere göre Price ortalamaları nedir?  
9. Şehir-Concept kırılımında Price ortalamaları nedir?  
10. `SaleCheckInDayDiff` değişkenini kategorik bir değişkene çeviriniz.  
11. Şehir-Concept-EB Score, Şehir-Concept-Seasons, Şehir-Concept-CInDay kırılımında ortalama ödenen ücret ve yapılan işlem sayısını inceleyiniz.  
12. City-Concept-Season kırılımının çıktısını Price’a göre sıralayınız ve `agg_df` olarak kaydediniz.  
13. İndekste yer alan isimleri değişken ismine çeviriniz.  
14. Yeni seviye tabanlı müşterileri (persona) tanımlayınız.  
15. Yeni müşterileri segmentlere ayırınız.  
16. Yeni gelen müşterileri sınıflandırıp, ne kadar gelir getirebileceklerini tahmin ediniz.  


## Öncelikle gerekli kütüphaneleri yükleyelim ve ayarlamaları yapalım:

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import warnings
warnings.filterwarnings('ignore')

## SORU1: miuul_gezinomi.xlsx dosyasını okutunuz ve veri setiyle ilgili genel bilgileri gösteriniz.

In [ ]:
df = pd.read_excel(r'C:\Users\ceren sahin\PycharmProjects\PYTHON_MIUUL\datasets\miuul_gezinomi.xlsx')

In [ ]:
df.head()

,SaleId,SaleDate,CheckInDate,Price,ConceptName,SaleCityName,CInDay,SaleCheckInDayDiff,Seasons
0,415122,2022-12-03,2022-12-03,79.30,Herşey Dahil,Antalya,Saturday,0,Low
1,415103,2022-12-03,2022-12-03,45.97,Yarım Pansiyon,Antalya,Saturday,0,Low
2,404034,2022-09-12,2022-09-13,77.84,Herşey Dahil,Antalya,Tuesday,1,High
3,415094,2022-12-03,2022-12-10,222.71,Yarım Pansiyon,İzmir,Saturday,7,Low
4,414951,2022-12-01,2022-12-03,140.48,Yarım Pansiyon,İzmir,Saturday,2,Low


In [ ]:
df.shape

(59164, 9)

In [ ]:
df.tail()

,SaleId,SaleDate,CheckInDate,Price,ConceptName,SaleCityName,CInDay,SaleCheckInDayDiff,Seasons
59159,51817,2016-01-05,2016-10-10,54.30,Herşey Dahil,Antalya,Monday,279,Low
59160,51816,2016-01-05,2016-10-10,54.30,Herşey Dahil,Antalya,Monday,279,Low
59161,51814,2016-01-05,2016-01-06,40.56,Herşey Dahil,Diğer,Wednesday,1,Low
59162,51736,2016-01-04,2016-01-05,69.85,Yarım Pansiyon,Diğer,Tuesday,1,Low
59163,51731,2016-01-04,2016-08-22,158.94,Herşey Dahil,Antalya,Monday,231,High


In [ ]:
df.index

RangeIndex(start=0, stop=59164, step=1)

In [ ]:
df.info()             #değişken bilgilerine baktım

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59164 entries, 0 to 59163
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   SaleId              59164 non-null  int64         
 1   SaleDate            59164 non-null  datetime64[ns]
 2   CheckInDate         59164 non-null  datetime64[ns]
 3   Price               59151 non-null  float64       
 4   ConceptName         59164 non-null  object        
 5   SaleCityName        59164 non-null  object        
 6   CInDay              59164 non-null  object        
 7   SaleCheckInDayDiff  59164 non-null  int64         
 8   Seasons             59164 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(2), object(4)
memory usage: 4.1+ MB


In [ ]:
df.describe().T       #sayısal değişkenlerin istatistiksel değerlerine baktım

,count,mean,min,25%,50%,75%,max,std
SaleId,59164.00,302675.80,51707.00,274038.50,321607.00,367373.00,415122.00,87943.22
SaleDate,59164,2021-02-11 17:13:11.751740928,2016-01-02 00:00:00,2020-08-19 00:00:00,2021-07-28 00:00:00,2022-05-10 00:00:00,2022-12-03 00:00:00,NaN
CheckInDate,59164,2021-03-15 11:23:01.732134400,2007-07-20 00:00:00,2020-08-24 00:00:00,2021-08-16 00:00:00,2022-07-04 00:00:00,2023-08-16 00:00:00,NaN
Price,59151.00,61.35,0.00,35.34,54.26,77.96,4880.47,51.38
SaleCheckInDayDiff,59164.00,31.84,0.00,2.00,10.00,34.00,630.00,51.49


In [ ]:
df.isnull().sum()     #boş değerlerin toplamına baktım

SaleId                 0
SaleDate               0
CheckInDate            0
Price                 13
ConceptName            0
SaleCityName           0
CInDay                 0
SaleCheckInDayDiff     0
Seasons                0
dtype: int64

In [ ]:
df.columns            #değişkenlere baktım

Index(['SaleId', 'SaleDate', 'CheckInDate', 'Price', 'ConceptName', 'SaleCityName', 'CInDay', 'SaleCheckInDayDiff', 'Seasons'], dtype='object')

## SORU2: kaç unique şehir vardır ve frekansları nedir?

In [ ]:
df['SaleCityName'].nunique()

6

In [ ]:
df['SaleCityName'].value_counts()          #frakanslarına buradan ulaşıyoruz

SaleCityName
Antalya    31649
Muğla      10662
Aydın      10646
Diğer       3245
İzmir       2507
Girne        455
Name: count, dtype: int64

## SORU3: kaç unique concept vardır?

In [ ]:
df['ConceptName'].nunique()

3

## SORU4: hangi concept'ten kaçar tane satış gerçekleşmiş?

In [ ]:
df['ConceptName'].value_counts()

ConceptName
Herşey Dahil      53186
Yarım Pansiyon     3559
Oda + Kahvaltı     2419
Name: count, dtype: int64

## SORU5: şehirlere göre satışlardan toplam ne kadar kazanılmış?

In [ ]:
df.groupby('SaleCityName')['Price'].sum()     #bu kodu da kullanabiliriz

SaleCityName
Antalya   2041911.10
Aydın      573296.01
Diğer      154572.29
Girne       27065.03
Muğla      665842.21
İzmir      165934.83
Name: Price, dtype: float64

In [ ]:
df.groupby('SaleCityName').agg({'Price':'sum'})     #bu kodu kullanmaya alışmak daha iyi

,Price
SaleCityName,
Antalya,2041911.10
Aydın,573296.01
Diğer,154572.29
Girne,27065.03
Muğla,665842.21
İzmir,165934.83


## SORU6: concept türlerine göre ne kadar kazanılmış?

In [ ]:
df.groupby('ConceptName').agg({'Price':'sum'})

,Price
ConceptName,
Herşey Dahil,3332910.77
Oda + Kahvaltı,121308.35
Yarım Pansiyon,174402.35


## SORU7: şehirlere göre price ortalamaları nedir?

In [ ]:
df.groupby('SaleCityName').agg({'Price':'mean'})

,Price
SaleCityName,
Antalya,64.52
Aydın,53.86
Diğer,47.71
Girne,59.48
Muğla,62.46
İzmir,66.27


## SORU8: conceptlere göre price ortalamaları nedir?

In [ ]:
df.groupby('ConceptName').agg({'Price':'mean'})

,Price
ConceptName,
Herşey Dahil,62.67
Oda + Kahvaltı,50.25
Yarım Pansiyon,49.03


## SORU9: şehir-concept kırılımında price ortalamaları nedir?

In [ ]:
df.groupby(by=['SaleCityName','ConceptName']).agg({'Price':'mean'})             #by kullanmasan da olur, birden fazla kırılım varsa kullanabilirsin

Price
SaleCityName ConceptName          
Antalya      Herşey Dahil    64.52
             Oda + Kahvaltı  63.50
             Yarım Pansiyon  67.19
Aydın        Herşey Dahil    54.00
             Oda + Kahvaltı  34.46
             Yarım Pansiyon  30.02
Diğer        Herşey Dahil    84.77
             Oda + Kahvaltı  37.60
             Yarım Pansiyon  42.11
Girne        Herşey Dahil    97.68
             Oda + Kahvaltı  39.78
             Yarım Pansiyon  53.25
Muğla        Herşey Dahil    63.02
             Oda + Kahvaltı  59.04
             Yarım Pansiyon  45.12
İzmir        Herşey Dahil    74.70
             Oda + Kahvaltı  41.32
             Yarım Pansiyon  59.61

## SORU10: SaleCheckInDayDiff değişkenini EB_Score adında yeni bir kategorik bir değişkene çeviriniz.  

  SaleCheckInDayDiff değişkeni müşterinin CheckIn tarihinden ne kadar önce satın alımını tamamladığını gösterir.  
  Aralıkları ikna edici şekilde oluşturunuz. Örneğin: '0_7','7_30','30_90','90_max' aralıklarını kullanabilirsiniz.  
  Bu aralıklar için 'Last Minuters', 'Potential Planners', 'Planners', 'Early Bookers' isimlerini kullanabilirsiniz.  

In [ ]:
df['SaleCheckInDayDiff'].head(10)

0    0
1    0
2    1
3    7
4    2
5    0
6    6
7    0
8    1
9    0
Name: SaleCheckInDayDiff, dtype: int64

In [ ]:
bins=[0,7,30,90,df['SaleCheckInDayDiff'].max()]
labels=['Last Minuters','Potential Planners','Planners','Early Bookers']
df['EB_Score'] = pd.cut(df['SaleCheckInDayDiff'], bins, labels=labels, include_lowest=True )
df['EB_Score'].value_counts()

EB_Score
Last Minuters         26564
Potential Planners    16621
Planners               9184
Early Bookers          6795
Name: count, dtype: int64

In [ ]:
df.head(50).to_excel('eb_scorew.xlsx', index=False)     #excel'e kaydettik

## SORU11: Şehir-Concept-EB Score, Şehir-Concept-Sezon, Şehir-Concept-ClnDay kırılımında ortalama ödenen ücret ve yapılan işlem sayısı cinsinden inceleyiniz.

In [ ]:
df.groupby(['SaleCityName','ConceptName','EB_Score']).agg({'Price':['mean','count']})

Price       
                                                mean  count
SaleCityName ConceptName    EB_Score                       
Antalya      Herşey Dahil   Last Minuters      62.75  14148
                            Potential Planners 64.90   8874
                            Planners           67.88   4490
                            Early Bookers      66.49   3281
             Oda + Kahvaltı Last Minuters      65.35    503
...                                              ...    ...
İzmir        Oda + Kahvaltı Early Bookers      66.93      3
             Yarım Pansiyon Last Minuters      57.79    636
                            Potential Planners 64.80    198
                            Planners           60.51     47
                            Early Bookers      65.91     14

[72 rows x 2 columns]

In [ ]:
df.groupby(['SaleCityName','ConceptName','Seasons']).agg({'Price':['mean','count']})

Price       
                                      mean  count
SaleCityName ConceptName    Seasons              
Antalya      Herşey Dahil   High     64.92  27126
                            Low      61.55   3667
             Oda + Kahvaltı High     66.27    303
                            Low      60.67    295
             Yarım Pansiyon High     73.26    118
                            Low      62.00    138
Aydın        Herşey Dahil   High     54.95  10103
                            Low      33.68    473
             Oda + Kahvaltı High     30.39     27
                            Low      44.45     11
             Yarım Pansiyon High     32.63     20
                            Low      25.27     11
Diğer        Herşey Dahil   High     83.79    364
                            Low      87.31    141
             Oda + Kahvaltı High     34.84    349
                            Low      39.96    408
             Yarım Pansiyon High     39.73    516
                            Low      42.95   1462
Girne        Herşey Dahil   High    103.94     55
                            Low      90.94     51
             Oda + Kahvaltı High     39.48     66
                            Low      40.04     73
             Yarım Pansiyon High     58.38    100
                            Low      48.58    110
Muğla        Herşey Dahil   High     63.38   9582
                            Low      48.30    237
             Oda + Kahvaltı High     61.26    592
                            Low      37.77     62
             Yarım Pansiyon High     45.82    177
                            Low      32.68     10
İzmir        Herşey Dahil   High     74.75   1234
                            Low      74.31    147
             Oda + Kahvaltı High     54.14     86
                            Low      33.56    142
             Yarım Pansiyon High     87.66    118
                            Low      55.35    777

In [ ]:
df.groupby(['SaleCityName','ConceptName','CInDay']).agg({'Price':['mean','count']})

Price      
                                       mean count
SaleCityName ConceptName    CInDay               
Antalya      Herşey Dahil   Friday    62.66  4136
                            Monday    63.26  6831
                            Saturday  64.42  4741
                            Sunday    65.85  3818
                            Thursday  62.89  3898
...                                     ...   ...
İzmir        Yarım Pansiyon Saturday  52.50   315
                            Sunday    98.93    68
                            Thursday  52.48    91
                            Tuesday   56.14    73
                            Wednesday 54.06    86

[126 rows x 2 columns]

## SORU12: City-Concept-Season kırılımının çıktısını Price'a göre sıralayınız.  

  Elde ettiğiniz çıktıyı agg_df olarak kaydediniz.

In [ ]:
agg_df = df.groupby(['SaleCityName','ConceptName','Seasons']).agg({'Price':'mean'}).sort_values('Price', ascending=False)
agg_df.head()

Price
SaleCityName ConceptName    Seasons       
Girne        Herşey Dahil   High    103.94
                            Low      90.94
İzmir        Yarım Pansiyon High     87.66
Diğer        Herşey Dahil   Low      87.31
                            High     83.79

## SORU13: indekste yer alan isimleri değişken ismine çeviriniz.  

  Üçüncü sorunun çıktısında yer alan price dışındaki tüm değişkenler index isimleridir. Bu isimleri değişken ismine çeviriniz.

In [ ]:
agg_df.reset_index(inplace=True)
agg_df.head()

,SaleCityName,ConceptName,Seasons,Price
0,Girne,Herşey Dahil,High,103.94
1,Girne,Herşey Dahil,Low,90.94
2,İzmir,Yarım Pansiyon,High,87.66
3,Diğer,Herşey Dahil,Low,87.31
4,Diğer,Herşey Dahil,High,83.79


## SORU14: yeni seviye tabanlı müşterileri (persona) tanımlayınız.  

  Yeni seviye tabanlı satışları tanımlayınız ve veri setine değişken olarak ekleyiniz.  
  Yeni eklenecek değişkenin adı: sales_level_based  
  Önceki soruda elde edeceğiniz çıktıdaki gözlemleri bir araya getirerek sales_level_based değişkenini oluşturmanız gerekmektedir.  

In [ ]:
agg_df['sales_level_based'] = (agg_df['SaleCityName'].str.upper() + '_' + agg_df['ConceptName'].str.upper() + '_' + agg_df['Seasons'].str.upper())

In [ ]:
#fonksiyonla yapalım:
agg_df['sales_level_based'] = agg_df[['SaleCityName','ConceptName','Seasons']].agg(lambda x: '_'.join(x).upper(), axis=1)
agg_df.head()

,SaleCityName,ConceptName,Seasons,Price,sales_level_based
0,Girne,Herşey Dahil,High,103.94,GIRNE_HERŞEY DAHIL_HIGH
1,Girne,Herşey Dahil,Low,90.94,GIRNE_HERŞEY DAHIL_LOW
2,İzmir,Yarım Pansiyon,High,87.66,İZMIR_YARIM PANSIYON_HIGH
3,Diğer,Herşey Dahil,Low,87.31,DIĞER_HERŞEY DAHIL_LOW
4,Diğer,Herşey Dahil,High,83.79,DIĞER_HERŞEY DAHIL_HIGH


## SORU15: yeni müşterileri (personaları) segmentlere ayırınız.  

  Yeni personaları price'a göre 4 segmente ayırınız.  
  Segmentleri segment isimlendirmesi ile değişken olarak agg_df'e ekleyiniz.  
  Segmentleri betimleyiniz. (segmentlere göre group by yapıp price mean, max, sum'larını alınız.)  

In [ ]:
agg_df['Segment'] = pd.qcut(agg_df['Price'], 4, labels=['D', 'C', 'B', 'A'])
agg_df.groupby('Segment').agg({'Price':['mean','max','sum']})

Price              
         mean    max    sum
Segment                    
D       33.37  39.48 300.30
C       44.89  54.14 403.99
B       60.27  64.92 542.47
A       82.47 103.94 742.21

## SORU16: yeni gelen müşterileri sınıflandırıp, ne kadar gelir getirebileceklerini tahmin ediniz.  

  Antalya'da her şey dahil ve yüksek sezonda tatil yapmak isteyen bir kişinin ortalama ne kadar gelir kazandırması beklenir?  
  Girne'de yarım pansiyon bir otele düşük sezonda giden bir tatilci hangi segmentte yer alacaktır?

In [ ]:
agg_df.sort_values(by='Price')
new_user = 'ANTALYA_HERŞEY DAHIL_HIGH'
agg_df[agg_df['sales_level_based'] == new_user]

,SaleCityName,ConceptName,Seasons,Price,sales_level_based,Segment
9,Antalya,Herşey Dahil,High,64.92,ANTALYA_HERŞEY DAHIL_HIGH,B


In [ ]:
new_user = 'GIRNE_YARIM PANSIYON_LOW'
agg_df[agg_df['sales_level_based'] == new_user]

,SaleCityName,ConceptName,Seasons,Price,sales_level_based,Segment
19,Girne,Yarım Pansiyon,Low,48.58,GIRNE_YARIM PANSIYON_LOW,C
